In [73]:
import math

import pandas as pd
import pandas_datareader as web
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import talib as ta

### install talib
1. [install talib](https://blog.quantinsti.com/install-ta-lib-python/)
2. ```pip3 install --upgrade  --no-binary numpy==1.20.1 numpy==1.20.1```

In [74]:
start = "2018-01-01"
end = "2020-01-01"

In [75]:
assets = ["FB", "AMZN", "AAPL", "NFLX", "GOOG"]

In [76]:
for asset in assets:
    exec("{} = web.DataReader(asset, data_source='yahoo', start = start, end = end)".format(asset))

In [77]:
asset_list = [FB, AMZN, AAPL, NFLX, GOOG]

In [78]:
#Calculating the technical indicators with talib using the functions for momentum indicators
def indikator(asset):
    asset["RSI"] = ta.RSI(asset["Close"], timeperiod = 14)
    asset["ROC"] = ta.ROC(asset["Close"], timeperiod = 10)
    asset["%R"]  = ta.WILLR(asset["High"], asset["Low"], asset["Close"], timeperiod = 14)
    asset["OBV"] = ta.OBV(asset["Close"], asset["Volume"])
    asset["MACD"], asset["MACD_SIGNAL"], asset["MACD_HIST"] = ta.MACD(asset["Close"], fastperiod=12, slowperiod=26, signalperiod=9)
    
    asset["Prediction"] = np.where(asset["Close"].shift(-5) > asset["Close"], 1, -1)

In [79]:
for asset in asset_list:
    indikator(asset)

In [80]:
df = FB.append((AMZN, AAPL, NFLX, GOOG))

In [81]:
df.dropna(inplace = True)

In [82]:
# Splitting the data into X and y variables and perform a train test split
from sklearn.model_selection import train_test_split

X = df[["RSI", "ROC", "OBV","%R", "MACD", "MACD_SIGNAL", "MACD_HIST"]]
y = df[["Prediction"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, train_size = 0.8, test_size=0.2)

In [83]:
# Creating the model with the Random Forest classifier, choosing the gini criterion
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators = 1000, oob_score = True, criterion = "gini", random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

C:\Users\eaut2\AppData\Local\Temp/ipykernel_22184/1557407622.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
c:\users\eaut2\.virtualenvs\flathunter-iemp75iw\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [84]:
print("Correct prediction in %: ", accuracy_score(y_test, y_pred, normalize= True)*100)

Correct prediction in %:  71.27659574468085


In [86]:
#Getting the Wallmart company for the timeperiod 2020 to 2021
test_stock = web.DataReader("WBT", data_source="yahoo", start="2020-01-01", end = "2021-02-14")

#Performing the indicator function on the new data
indikator(test_stock)

#Dropping all NaN values so the model can make predictions on the new data
test_stock.dropna(inplace=True)

In [ ]:
test_stock["predicted"] = model.predict(test_stock[["RSI", "ROC", "OBV","%R", "MACD", "MACD_SIGNAL", "MACD_HIST"]])

In [ ]:
test_stock.head()

In [ ]:
def buy_sell(asset):
    BuyPrice = []
    SellPrice = []
    flag = -1
    counter = 0
    n=5
    
    for i in range(len(asset)):
        if asset["predicted"][i] == 1 and counter == 0:
            if flag != 1:
                BuyPrice.append(asset["Adj Close"][i])
                SellPrice.append(np.nan)
                flag = 1
            else:
                BuyPrice.append(np.nan)
                SellPrice.append(np.nan)
        elif asset["predicted"][i] == -1 and counter ==0:
            if flag !=0:
                BuyPrice.append(np.nan)
                SellPrice.append(asset["Adj Close"][i])
                flag = 0
            else:
                BuyPrice.append(np.nan)
                SellPrice.append(np.nan)
        else:
            BuyPrice.append(np.nan)
            SellPrice.append(np.nan)
            
        counter +=1
        if counter ==n:
            counter = 0
            
    return (BuyPrice, SellPrice)

In [ ]:
buysell = buy_sell(test_stock)

In [ ]:
test_stock["Buy"] = buysell[0]
test_stock["Sell"] = buysell[1]

In [ ]:
#Replacing the first Sell signal by a NaN value
test_stock["Sell"][0] = np.nan

In [ ]:
#Plotting our Signals on our time period
plt.figure(figsize=(16,8))
plt.scatter(test_stock.index, test_stock["Buy"], color = "green", label = "Buy", marker = "^", alpha=1)
plt.scatter(test_stock.index, test_stock["Sell"], color = "red", label = "Sell", marker = "v", alpha=1)
plt.plot(test_stock.index, test_stock["Adj Close"], alpha = 0.5)
plt.legend()
plt.grid()
plt.show()

In [ ]:
def performance(data):
    performance = [1000000]
    num_stocks = 0
    for i in range(len(data)):
        if math.isnan(data["Buy"][i]) == False:
            num_stocks = performance[i]/data["Adj Close"][i]
            performance.append(num_stocks*data["Adj Close"][i])
        elif math.isnan(data["Sell"][i]) == False:
            cash = num_stocks * data["Adj Close"][i]
            performance.append(cash)
        else: 
            performance.append(performance[i])
            
    return performance

In [ ]:
test_stock_performance = performance(test_stock)

In [ ]:
plt.figure(figsize= (12.6, 4.6))
plt.plot(test_stock_performance)
plt.grid()
plt.show()